In [9]:
import pyscf
import pyscf.tools

In [10]:
molecule = """
 Fe 1.67785607 0.00052233 0.06475932
 Fe -1.67785607 -0.00052233 0.06475932
 O 0.00000000 0.00000000 -0.47099074
 Cl 1.87002704 -1.09796437 1.99091682
 Cl 2.93244917 -0.98210488 -1.47467288
 Cl 2.37160936 2.07954091 -0.50446591
 Cl -1.87002704 1.09796437 1.99091682
 Cl -2.93244917 0.98210488 -1.47467288
 Cl -2.37160936 -2.07954091 -0.50446591
"""

In [12]:
basis = "def2-svp"
pymol = pyscf.gto.Mole(
        atom    =   molecule,
        symmetry=   True,
        spin    =   10, # number of unpaired electrons
        charge  =   -2,
        basis   =   basis)


pymol.build()
print("symmetry: ",pymol.topgroup)
# mf = pyscf.scf.UHF(pymol).x2c()
mf = pyscf.scf.ROHF(pymol)
# mf = pyscf.scf.RHF(pymol)
mf.verbose = 4
mf.conv_tol = 1e-8
mf.conv_tol_grad = 1e-5
mf.chkfile = "scf.fchk"
mf.init_guess = "sad"
mf.run(max_cycle=200)

print(" Hartree-Fock Energy: %12.8f" % mf.e_tot)
# mf.analyze()

symmetry:  C2


******** <class 'pyscf.scf.hf_symm.SymAdaptedROHF'> ********
method = SymAdaptedROHF-ROHF-RHF
initial guess = sad
damping factor = 0
level_shift factor = 0
DIIS = <class 'pyscf.scf.diis.CDIIS'>
diis_start_cycle = 1
diis_space = 8
SCF conv_tol = 1e-08
SCF conv_tol_grad = 1e-05
SCF max_cycles = 200
direct_scf = True
direct_scf_tol = 1e-13
chkfile to save SCF result = scf.fchk
max_memory 4000 MB (current use 0 MB)
num. doubly occ = 77  num. singly occ = 10
init E= -5351.41478186842
HOMO (B) = 0.318236980196613  LUMO (A) = 0.320979897364177
cycle= 1 E= -5347.0130535218  delta_E=  4.4  |g|= 4.93  |ddm|= 11.8
HOMO (B) = -0.208577216881654  LUMO (A) = 0.00879930358562003
cycle= 2 E= -5326.86878641886  delta_E= 20.1  |g|= 8.88  |ddm|= 13.1
HOMO (B) = 1.25634475236875  LUMO (B) = 0.0701291625721957
cycle= 3 E= -5328.84109730652  delta_E= -1.97  |g|= 5.69  |ddm|= 29.9
HOMO (A) = 0.189978800807605  LUMO (B) = 0.322966982029677
cycle= 4 E= -5355.41569900171  delta_E= -26.6  |g|= 0.

In [13]:
F = mf.get_fock()

In [43]:
X = scipy.linalg.sqrtm(S)
Xinv = scipy.linalg.inv(X)
Forth = Xinv @ F @ Xinv.T
e,Corth = np.linalg.eigh(Forth)
docc_inds = []
socc_inds = []
virt_inds = []
for i,ei in enumerate(e):
    print(" %4i %14.8f %14.8f %5i" %(i,ei, ei-mf.mo_energy[i], mf.mo_occ[i]))
    if mf.mo_occ[i] == 2:
        docc_inds.append(i)
    elif mf.mo_occ[i] == 1:
        socc_inds.append(i)
    elif mf.mo_occ[i] == 0:
        virt_inds.append(i)
    else:
        error()
    
Docc = Corth[:,docc_inds]
Socc = Corth[:,socc_inds]
Virt = Corth[:,virt_inds]

    0  -261.34809709     0.00000360     2
    1  -261.34809101     0.00000360     2
    2  -104.49623551    -0.00000041     2
    3  -104.49622754    -0.00000041     2
    4  -104.46882198    -0.00000119     2
    5  -104.46881851    -0.00000122     2
    6  -104.46869102     0.00000108     2
    7  -104.46867655     0.00000111     2
    8   -31.92654817     0.00000343     2
    9   -31.92654471     0.00000343     2
   10   -27.41495120     0.00000345     2
   11   -27.41494930     0.00000345     2
   12   -27.41410158     0.00000349     2
   13   -27.41409796     0.00000349     2
   14   -27.41325436     0.00000344     2
   15   -27.41320326     0.00000344     2
   16   -20.21426707    -0.00000209     2
   17   -10.26048678    -0.00000032     2
   18   -10.26048639    -0.00000032     2
   19   -10.23363499     0.00000102     2
   20   -10.23363218     0.00000102     2
   21   -10.23282989    -0.00000106     2
   22   -10.23282965    -0.00000106     2
   23    -7.73497211    -0.0000003

In [36]:
import numpy as np
import scipy
import copy as cp
import math

def get_frag_bath(Pin, frag, S, thresh=1e-7, verbose=2):
    print(" Frag: ", frag)
    X = scipy.linalg.sqrtm(S)
    Xinv = scipy.linalg.inv(X)

    Nbas = S.shape[0]
    Cfrag = Xinv@np.eye(Nbas)[:,frag]
    

    P = X@Pin@X.T

    nfrag = np.trace(P[frag,:][:,frag])
    P[frag,:] = 0
    P[:,frag] = 0
    bath_idx = []
    env_idx = []
    e,U = np.linalg.eigh(P)
    nbath = 0.0
    for nidx,ni in enumerate(e):
        if math.isclose(ni, 1, abs_tol=thresh):
            env_idx.append(nidx)
        elif thresh < ni < 1-thresh:
            if verbose > 1:
                print(" eigvalue: %12.8f" % ni)
            bath_idx.append(nidx)
            nbath += ni
        
            
    print(" # Electrons frag: %12.8f bath: %12.8f total: %12.8f" %(nfrag, nbath, nfrag+nbath))
    Cenv = Xinv@U[:,env_idx]
    Cbath = Xinv@U[:,bath_idx]
    
    
    C = np.hstack((Cfrag, Cbath))
    
    # Get virtual orbitals (these are just the orthogonal complement of the env and frag/bath
    Q = np.eye(C.shape[0]) - X@C@C.T@X - X@Cenv@Cenv.T@X
    e,U = np.linalg.eigh(Q)
    vir_idx = []
    for nidx,ni in enumerate(e):
        if math.isclose(ni, 1, abs_tol=thresh):
            vir_idx.append(nidx)
    Cvir = Xinv@U[:,vir_idx]

    assert(Cenv.shape[1] + Cvir.shape[1] + C.shape[1] == Nbas)

          
    # print(C.T@S@C)
    return (Cenv, C, Cvir)

def gram_schmidt(frags, S, thresh=1e-8):
    # |v'> = (1-sum_i |i><i|) |v>
    #      = |v> - sum_i |i><i|v>
    Nbas = S.shape[1]
    seen = []
    out = []
    seen = np.zeros((Nbas, 0))

    for f in frags:
        outf = np.zeros((Nbas, 0))
        # grab each orbital
        for fi in range(f.shape[1]):
            v = f[:,fi]
            v.shape = (Nbas, 1)

            # Compare to previous orbitals
            for fj in range(seen.shape[1]):
                j = seen[:,fj]
                j.shape = (Nbas, 1)
                ovlp = (j.T @ S @ v)[0]
                v = v - j * ovlp

                norm = np.sqrt((v.T @ S @ v)[0])
                if norm < thresh:
                    print(" Warning: small norm in GS: ", norm)
                v = v/norm
            
            outf = np.hstack((outf, v))
            seen = np.hstack((seen, v))
        out.append(outf)
    return out

def sym_ortho(frags, S, thresh=1e-8):
    Nbas = S.shape[1]
    
    inds = []
    Cnonorth = np.hstack(frags)
    shift = 0
    for f in frags:
        inds.append(list(range(shift, shift+f.shape[1])))
        shift += f.shape[1]
        
    
    Smo = Cnonorth.T @ S @ Cnonorth
    X = np.linalg.inv(scipy.linalg.sqrtm(Smo))
    # print(Cnonorth.shape, X.shape)
    Corth = Cnonorth @ X
    
    frags2 = []
    for f in inds:
        frags2.append(Corth[:,f])
    return frags2


In [96]:
import scipy

# Find AO's corresponding to atoms 
# print(mf.mol.aoslice_by_atom())
# print(mf.mol.ao_labels(fmt=False, base=0))
full = []
basis_Fe1 = []
basis_Fe2 = []
basis_O2sp = []
basis_Fe4d = []
frag1 = []
frag2 = []
frag3 = []
for ao_idx,ao in enumerate(mf.mol.ao_labels(fmt=False)):
    if ao[0] == 0:
        basis_Fe1.append(ao_idx)
        if ao[2] in ("4d"):
            basis_Fe4d.append(ao_idx)
    elif ao[0] == 1:
        basis_Fe2.append(ao_idx)
        if ao[2] in ("4d"):
            basis_Fe4d.append(ao_idx)
    elif ao[0] == 2:
        if ao[2] in ("2s", "2p"):
            basis_O2sp.append(ao_idx)


tmp = np.zeros(Docc.shape)
for i in basis_O2sp:
    tmp[i,:] = Docc[i,:]

Udocc,sdocc,Vdocc = np.linalg.svd(tmp, full_matrices=False)
# for i,si in enumerate(sdocc):
#     print(si)
Vdocc = Vdocc[0:4,:]
C_O = Docc @ Vdocc[0:4,:].T
C_env = Docc @ Vdocc[4::,:].T

tmp = np.zeros(Virt.shape)
for i in basis_Fe4d:
    tmp[i,:] = Virt[i,:]

Udocc,sdocc,Vdocc = np.linalg.svd(tmp, full_matrices=False)
# for i,si in enumerate(sdocc):
#     print(si)
C_Fe = Virt @ Vdocc[0:len(basis_Fe4d),:].T

Cact = np.hstack((C_O, Socc, C_Fe))
print(Cact.shape)
print(" Here: ", np.linalg.norm(Cact.T@C_env))


tmp1 = np.zeros(Cact.shape)
tmp2 = np.zeros(Cact.shape)
for i in basis_Fe1:
    tmp1[i,:] = Cact[i,:]
for i in basis_Fe2:
    tmp2[i,:] = Cact[i,:]

U1, s1, V1 = np.linalg.svd(tmp1, full_matrices=False) 
U2, s2, V2 = np.linalg.svd(tmp2, full_matrices=False) 

C1 = Cact @ V1[0:12,:].T
C2 = Cact @ V2[0:12,:].T

# for si in s1:
#     print(si)
    
    
C1 = Xinv @ C1[:,0:12]
C2 = Xinv @ C2[:,0:12]
frag_orbs = sym_ortho((C1, C2), S)
# frag_orbs = gram_schmidt((C1, C2), S)

# Cact = Xinv@Cact
# # pyscf.tools.molden.from_mo(mf.mol, "C_act.molden", np.hstack((C1,C2)))

print(" Here: ", np.linalg.norm(Cact.T@Cenv))

Nbas = S.shape[1]
Ctot = np.zeros((Nbas,0))

clusters = []
init_fspace = []

ci_shift = 0
for fi,f in enumerate(frag_orbs):

    assert np.linalg.norm(f.T @ C_env) < 1e-12 

    # Canonicalize
    focki = f.T @ F @ f
    ei,vi = np.linalg.eigh(focki)
    for ei_idx,e in enumerate(ei):
        print(" %4i %12.8f"%(ei_idx, e))
    
    
    f = f@vi
    P = mf.make_rdm1()
    Pa = P[0,:,:]
    Pb = P[1,:,:]
    
    Ctot = np.hstack((Ctot, f))
    Paf = f.T @ S @ Pa @ S @ f
    Pbf = f.T @ S @ Pb @ S @ f
    
    na = np.trace(Paf)
    nb = np.trace(Pbf)
    
    clusters.append(list(range(ci_shift, ci_shift+f.shape[1])))
    ci_shift += f.shape[1]
    init_fspace.append((int(np.round(na)), int(np.round(nb))))
    print(" Fragment: %4i %s" %(fi,frags[fi]))
    print("    # α electrons: %12.8f" % na)
    print("    # β electrons: %12.8f" % nb)

    # pyscf.tools.molden.from_mo(mf.mol, "C_frag%2i.molden"%fi, f);

    
pyscf.tools.molden.from_mo(mf.mol, "C_act.molden", Ctot);



(184, 24)
 Here:  0.0
 Here:  3.363164405368187
    0  -1.43591133
    1  -0.19998831
    2  -0.08405763
    3  -0.07897519
    4  -0.06846718
    5  -0.05577903
    6  -0.03869914
    7   1.50027849
    8   1.52764328
    9   1.61463460
   10   1.70630435
   11   1.76484909
 Fragment:    0 []
    # α electrons:   7.00000000
    # β electrons:   2.00000009
    0  -1.43591133
    1  -0.19998831
    2  -0.08405763
    3  -0.07897519
    4  -0.06846718
    5  -0.05577903
    6  -0.03869914
    7   1.50027849
    8   1.52764328
    9   1.61463460
   10   1.70630435
   11   1.76484909
 Fragment:    1 []
    # α electrons:   7.00000000
    # β electrons:   2.00000009

WARN: orbitals [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23] not symmetrized, norm = [0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5
 0.5 0.5 0.5 0.5 0.5 0.5]



In [97]:
print(clusters)
print(init_fspace)

[[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11], [12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23]]
[(7, 2), (7, 2)]


# Make Integrals

In [98]:
d1_embed = 2 * C_env @ C_env.T

h0 = pyscf.gto.mole.energy_nuc(mf.mol)
h  = pyscf.scf.hf.get_hcore(mf.mol)
j, k = pyscf.scf.hf.get_jk(mf.mol, d1_embed, hermi=1)

In [99]:
h0 += np.trace(d1_embed @ ( h + .5*j - .25*k))

h = Ctot.T @ h @ Ctot;
j = Ctot.T @ j @ Ctot;
k = Ctot.T @ k @ Ctot;

In [100]:
nact = h.shape[0]

h2 = pyscf.ao2mo.kernel(pymol, Ctot, aosym="s4", compact=False)
h2.shape = (nact, nact, nact, nact)

In [101]:
# The use of d1_embed only really makes sense if it has zero electrons in the
# active space. Let's warn the user if that's not true

S = pymol.intor("int1e_ovlp_sph")
n_act = np.trace(S @ d1_embed @ S @ Ctot @ Ctot.T)
if abs(n_act) > 1e-8 == False:
    print(n_act)
    error(" I found embedded electrons in the active space?!")

h1 = h + j - .5*k;


In [102]:
np.save("ints_h0", h0)
np.save("ints_h1", h1)
np.save("ints_h2", h2)
np.save("mo_coeffs", Ctot)
np.save("overlap_mat", S)

In [103]:
np.trace(Ctot @ Ctot.T @ F)

12.303663966573888

In [104]:
np.trace(Cact @ Cact.T @ F)

-1.4249419675738506